In [1]:
from landmark_detection.pipeline import Pipeline_Landmark_Detection
from landmark_detection.utils import show_image, show_bboxes, show_similarity_search
from benchmark.revisitop.evaluate import compute_map
from benchmark.revisitop.dataset import configdataset
from benchmark.evaluation import run_evaluation, run_evaluation2, show_inference_example

import os
import pandas as pd
import pickle
import numpy as np
import torch

In [4]:
VERSION = 0

force_rebuild = True
save_every = 500
config = {
    'version': VERSION,
    
    # DATABASE
    'min_area': 0.0,
    'min_sim_db': 0.0,
    
    # PIPELINE
    'detector_file': "yolov8n-oiv7.pt",
    'extractor_onnx_file': "cvnet-sg-v" + str(VERSION) + ".onnx",
    'pipeline_onnx_file': "pipeline-yolo-cvnet-sg-v" + str(VERSION) + ".onnx",
    'image_dim': (640, 640),
    'allowed_classes': [41,68,70,74,87,95,113,144,150,158,164,165,193,205,212,224,257,
                                  298,310,335,351,354,390,393,401,403,439,442,457,466,489,510,512,
                                  514,524,530,531,543,546,554,565,573,580,588,591],
    'score_thresh': 0.05,
    'iou_thresh': 0.25,
    'scales': [0.7071, 1.0, 1.4142],
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
    'rgem_pr': 2.5,
    'rgem_size': 5,
    'gem_p': 4.6,
    'sgem_ps': 10.0,
    'sgem_infinity': False,
    'eps': 1e-8,
}
topk = 10
min_sim = 0.70
min_votes = 0.60
remove_inner_boxes = 0.5
join_boxes = True

In [6]:
# Fixed Paths
CLASS_NAMES_PATH = os.path.join("oiv7.yaml")
DATASETS_PATH = os.path.abspath("datasets")
ROXFORD5K_PATH = os.path.join(DATASETS_PATH, "roxford5k", "jpg")
ROXFORD5K_PKL = os.path.join(DATASETS_PATH, "roxford5k", "gnd_roxford5k.pkl")
ROXFORD5K_CSV = os.path.join(DATASETS_PATH, "roxford5k", "roxford5k_image_data.csv")

In [7]:
cfg = configdataset('roxford5k', DATASETS_PATH)

In [8]:
PIPELINE_CONFIG_PATH = os.path.join("configs", "pipeline-yolo-cvnet-sg-v" + str(VERSION))

# Dataset Oxford
ROXFORD5K_DF = os.path.join(DATASETS_PATH, "roxford5k", "results", "df_roxford5k-v" + str(VERSION) + ".pkl")
ROXFORD5K_DESC = os.path.join(DATASETS_PATH, "roxford5k", "results", "desc_roxford5k-v" + str(VERSION) + ".pkl")

In [ ]:
pipeline = Pipeline_Landmark_Detection(
        detector_file = config['detector_file'],
        extractor_onnx_file = config['extractor_onnx_file'],
        pipeline_onnx_file = config['pipeline_onnx_file'],
        image_dim = config['image_dim'],
        allowed_classes = config['allowed_classes'],
        score_thresh = config['score_thresh'],
        iou_thresh = config['iou_thresh'],
        scales = config['scales'],
        mean = config['mean'],
        std = config['std'],
        rgem_pr = config['rgem_pr'],
        rgem_size = config['rgem_size'],
        gem_p = config['gem_p'],
        sgem_ps = config['sgem_ps'],
        sgem_infinity = config['sgem_infinity'],
        eps = config['eps'],
        topk = topk,
        min_sim = min_sim,
        min_votes = min_votes,
        remove_inner_boxes = remove_inner_boxes,
        join_boxes = join_boxes
)
pipeline.to_json(PIPELINE_CONFIG_PATH)

In [ ]:
# Build/load roxford5k dataset
df_result_ox, descriptors_final_ox, places_db_ox = pipeline.build_image_database(
    image_folder = ROXFORD5K_PATH,
    df_pickle_path = ROXFORD5K_DF,
    descriptor_pickle_path = ROXFORD5K_DESC,
    return_places_db = True,
    force_rebuild = force_rebuild,
    save_every = save_every,
    min_area = min_area,
    min_sim = min_sim_db
)

In [ ]:
# Evaluate roxford5k
roxford5k_eval = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k')

In [ ]:
# Evaluate roxford5k with bbox
roxford5k_eval_bbox = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k', use_bbox=True)